In [ ]:
!pip install pandarallel

  Created wheel for pandarallel: filename=pandarallel-1.5.2-py3-none-any.whl size=18383 sha256=84f26e245b6f40693ffe08b6b617a40f556fbb3e92c4cdf49a7e66a8fb575ddb
  Stored in directory: /root/.cache/pip/wheels/b5/6d/51/9ece2eaf007ea3f7fb0ce053c5773b2eb0d308887da3af12c1
Successfully built pandarallel
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [ ]:
import numpy as np
import pandas as pd
import os
import random
import torch
import pickle
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel, AdamW
from torch import nn
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import Dataset
from torch.optim import Adam
from torch.nn.utils import clip_grad_norm_
from pandarallel import pandarallel
from IPython.display import clear_output
from tqdm import tqdm

In [ ]:
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
df = pd.read_pickle('df.pkl')

In [ ]:
df

,text,toxic?,dataset
0,Explanation\nWhy the edits made under my usern...,0,0
1,D'aww! He matches this background colour I'm s...,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0
3,"""\nMore\nI can't make any real suggestions on ...",0,0
4,"You, sir, are my hero. Any chance you remember...",0,0
...,...,...,...
281274,@USER @USER Nothing about trump is human or no...,1,4
281275,@USER Oh shit. That sounds like a cool time tho,1,4
281276,@USER @USER I gotta say that shit to you one t...,1,4
281277,@USER tRUmp does everything he can to destroy ...,1,4


In [ ]:
df['toxic?'].mean()

0.18884097284191143

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tqdm.pandas()
encoded_info = df['text'].progress_apply(tokenizer.encode_plus,
                                         add_special_tokens = True).values
df['tokens'] = [x['input_ids'] for x in encoded_info]
df.drop(df[df['tokens'].map(len) > 512].index, inplace=True)

df.shape

/opt/conda/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
  0%|          | 41/281279 [00:00<11:55, 393.30it/s]

100%|██████████| 281279/281279 [09:45<00:00, 480.32it/s] 


(276143, 4)

In [ ]:
df

,text,toxic?,dataset,tokens
0,Explanation\nWhy the edits made under my usern...,0,0,"[101, 7526, 2339, 1996, 10086, 2015, 2081, 210..."
1,D'aww! He matches this background colour I'm s...,0,0,"[101, 1040, 1005, 22091, 2860, 999, 2002, 3503..."
2,"Hey man, I'm really not trying to edit war. It...",0,0,"[101, 4931, 2158, 1010, 1045, 1005, 1049, 2428..."
3,"""\nMore\nI can't make any real suggestions on ...",0,0,"[101, 1000, 2062, 1045, 2064, 1005, 1056, 2191..."
4,"You, sir, are my hero. Any chance you remember...",0,0,"[101, 2017, 1010, 2909, 1010, 2024, 2026, 5394..."
...,...,...,...,...
281274,@USER @USER Nothing about trump is human or no...,1,4,"[101, 1030, 5310, 1030, 5310, 2498, 2055, 8398..."
281275,@USER Oh shit. That sounds like a cool time tho,1,4,"[101, 1030, 5310, 2821, 4485, 1012, 2008, 4165..."
281276,@USER @USER I gotta say that shit to you one t...,1,4,"[101, 1030, 5310, 1030, 5310, 1045, 10657, 236..."
281277,@USER tRUmp does everything he can to destroy ...,1,4,"[101, 1030, 5310, 8398, 2515, 2673, 2002, 2064..."


In [ ]:
class ToxicityDataset(Dataset):
    def __init__(self, df, batch_size):
        self.df = df
        self.batch_size = batch_size
        self.num_samples = df.shape[0] 

    def __len__(self):
        return df.shape[0] // self.batch_size

    def __getitem__(self, index):
        batch = self.df.iloc[self.batch_size * index:
                             min(self.num_samples, self.batch_size * (index + 1))]
        num_tokens = [len(sample) for sample in batch['tokens'].tolist()]
        
        padded_tokens = torch.tensor(np.matrix(
            pad_sequences(batch['tokens'].tolist(), maxlen=512, padding='post')
        )).type(torch.LongTensor)
        masks = torch.tensor([[1 for _ in range(x)] + [0 for _ in range(512 - x)] for x in num_tokens])
        labels = torch.tensor(batch['toxic?'].astype(int).values.tolist()).type(torch.FloatTensor)

        return padded_tokens, masks, labels
    
    def shuffle(self):
        self.df = self.df.sample(frac=1).reset_index(drop=True)

batch_size = 16
dataset = ToxicityDataset(df[['tokens', 'toxic?']], batch_size=batch_size)

In [ ]:
class BertClassifier(nn.Module):
    
    def __init__(self, bert):
        super().__init__()
        self.bert = bert
        self.hidden = nn.Linear(768, 64)
        self.activation = nn.LeakyReLU(0.1)
        self.output = nn.Linear(64, 1)
        
    def forward(self, input_ids, attention_mask, labels=None):
        
        _, cls_output = self.bert(input_ids, attention_mask=attention_mask)
        cls_output = self.hidden(cls_output)
        cls_output = self.activation(cls_output)
        cls_output = self.output(cls_output)
        cls_output = torch.sigmoid(cls_output)
        criterion = nn.BCELoss()
        loss = 0
        if labels is not None:
            loss = criterion(cls_output[:, 0], labels.float())
        return loss, cls_output

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertClassifier(BertModel.from_pretrained('bert-base-uncased')).to(device)

In [ ]:
print(model)

BertClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

In [ ]:
optimizer = AdamW(model.parameters(), lr=3e-6)
if device.type == 'gpu':
    torch.cuda.empty_cache()

In [ ]:
epochs = 1
n_batches = df.shape[0] // batch_size
batch_losses = []

for epoch_num in range(epochs):
    dataset.shuffle()
    model.train()
    train_loss = 0
    for step_num in tqdm(range(len(dataset))):
        batch_data = dataset[step_num]
        token_ids, masks, labels = tuple(t.to(device) for t in batch_data)

        loss, _ = model(input_ids=token_ids, attention_mask=masks, labels=labels)

        train_loss += loss.item()
        
        model.zero_grad()
        loss.backward()        

        clip_grad_norm_(parameters=model.parameters(), max_norm=1.0)
        optimizer.step()
        
        batch_losses.append(train_loss / (step_num + 1))

100%|██████████| 17258/17258 [4:14:39<00:00,  1.13it/s]


In [ ]:
with open('loss.pkl', 'wb') as f:
    pickle.dump(batch_losses, f)

In [ ]:
model.bert.save_pretrained('bert_model')